In [27]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from pathlib import Path


In [28]:
file ='AI_Powered_Eduction_system'
def root(root_name:str=file):
    for parent in Path().resolve().parents:
        if parent.name == root_name:
            return(parent) 
ROOT = root()

In [29]:
con = sql.connect(ROOT/r'data\Train_datasets\Pass_Fail\AI_Edu_system.db')
cur = con.cursor()

In [30]:
df_data = '''SELECT * FROM student_log'''

In [31]:
student_log = pd.read_sql('''SELECT * FROM student_log''',con)

In [32]:
print(len(student_log))

881035


In [33]:
sample_log = student_log.sample(n=100000,random_state=1)

In [34]:
X_cols = list(sample_log.columns)
X_cols.remove('correct')
Y_col = ['correct']


In [35]:
X = sample_log[X_cols]
y=sample_log[Y_col]

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [37]:
print('n_1s:',np.sum(y_train==1))
print('n_0s:',np.sum(y_train==0))

n_1s: correct    26656
dtype: int64
n_0s: correct    43344
dtype: int64


c:\Users\Udith\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [38]:
from imblearn.under_sampling import RandomUnderSampler

In [39]:
rus = RandomUnderSampler(random_state=1)
x_train_resampled,y_train_resampled=rus.fit_resample(x_train,y_train)

In [40]:
print('n_1s:',np.sum(y_train_resampled==1))
print('n_0s:',np.sum(y_train_resampled==0))

n_1s: correct    26656
dtype: int64
n_0s: correct    26656
dtype: int64


c:\Users\Udith\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
m = LogisticRegression()
m.fit(x_train_resampled,y_train_resampled)


c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [43]:
y_pred=m.predict(x_test)

In [44]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.55      0.63     18614
           1       0.48      0.68      0.57     11386

    accuracy                           0.60     30000
   macro avg       0.61      0.62      0.60     30000
weighted avg       0.64      0.60      0.61     30000



In [46]:
m = LogisticRegression(penalty='elasticnet',C=1,solver='saga',l1_ratio=1)
m.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=1, l1_ratio=1, penalty='elasticnet', solver='saga')

In [47]:
y_pred=m.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.52      0.60     18614
           1       0.45      0.65      0.54     11386

    accuracy                           0.57     30000
   macro avg       0.58      0.59      0.57     30000
weighted avg       0.61      0.57      0.58     30000



In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
rfc1=RandomForestClassifier()
rfc1.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [50]:
y_pred=rfc1.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.63      0.71     18614
           1       0.56      0.76      0.64     11386

    accuracy                           0.68     30000
   macro avg       0.68      0.69      0.68     30000
weighted avg       0.71      0.68      0.68     30000



In [51]:
import pickle

In [52]:
filename=ROOT/r'models/correct_predictor.sav'
pickle.dump(rfc1,open(filename,'wb'))

In [53]:
rfc=RandomForestClassifier(max_depth=4)
rfc.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=4)

In [54]:
y_pred=rfc.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.56      0.67     18614
           1       0.53      0.82      0.64     11386

    accuracy                           0.66     30000
   macro avg       0.68      0.69      0.66     30000
weighted avg       0.72      0.66      0.66     30000



In [55]:
rfc=RandomForestClassifier(max_depth=10)
rfc.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10)

In [56]:
rfc.score(x_train_resampled,y_train_resampled)

0.7411839735894358

In [57]:
y_pred=rfc.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.53      0.67     18614
           1       0.54      0.89      0.67     11386

    accuracy                           0.67     30000
   macro avg       0.72      0.71      0.67     30000
weighted avg       0.76      0.67      0.67     30000



In [58]:
rfc=RandomForestClassifier(n_estimators=500)
rfc.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=500)

In [59]:
y_pred=rfc.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.63      0.71     18614
           1       0.56      0.76      0.64     11386

    accuracy                           0.68     30000
   macro avg       0.68      0.70      0.68     30000
weighted avg       0.72      0.68      0.68     30000



In [60]:
from sklearn.svm import LinearSVC

In [61]:
m=LinearSVC(max_iter=10000,C=1.9)
m.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.9, max_iter=10000)

In [62]:
y_pred=m.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.53      0.63     18614
           1       0.50      0.75      0.60     11386

    accuracy                           0.62     30000
   macro avg       0.64      0.64      0.62     30000
weighted avg       0.67      0.62      0.62     30000



In [63]:
from sklearn.svm import SVC

In [64]:
m1=SVC(kernel='linear',max_iter=1000)
m1.fit(x_train_resampled,y_train_resampled)

c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Udith\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(kernel='linear', max_iter=1000)

In [65]:
y_pred=m1.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.63      0.37      0.47     18614
           1       0.38      0.64      0.48     11386

    accuracy                           0.47     30000
   macro avg       0.51      0.51      0.47     30000
weighted avg       0.54      0.47      0.47     30000

